In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("HELK Reader") \
    .master("spark://helk-spark-master:7077") \
    .enableHiveSupport() \
    .getOrCreate()

In [3]:
es_reader = (spark.read
    .format("org.elasticsearch.spark.sql")
    .option("inferSchema", "true")
    .option("es.read.field.as.array.include", "tags")
    .option("es.nodes","helk-elasticsearch:9200")
    .option("es.net.http.auth.user","elastic")
)

In [4]:
sysmon_df = es_reader.load("logs-endpoint-winevent-sysmon-*/")

In [5]:
sysmon_df.createOrReplaceTempView("sysmon_events")

In [32]:
Hook_Injection = spark.sql(
'''
SELECT 
    b.host_name,
    b.process_parent_name,
    b.process_name,
    a.process_granted_access,
    a.user_reporter_name
FROM sysmon_events b
JOIN sysmon_events a
    ON a.process_target_guid = b.process_guid
    AND a.event_id = 10
    AND a.process_granted_access = "2097151"
    AND b.process_parent_name = a.process_name
    AND a.process_name != b.process_name
JOIN sysmon_events c
    ON b.process_guid = c.process_guid
    AND c.event_id = 13
    AND c.process_guid = a.process_target_guid
    AND c.process_name = a.process_target_name 
    AND (substring_index(c.registry_key_path, '\\\\', -1)) = b.process_name
JOIN sysmon_events d
    ON d.process_guid = a.process_target_guid
    AND d.event_id = 7
    AND d.module_loaded = a.process_target_path
    AND LOWER(d.OriginalFileName) = (substring_index(c.registry_key_path, '\\\\', -1))
    AND d.OriginalFileName = b.OriginalFileName
WHERE
     b.event_id = 1
    AND b.process_integrity_level = "High"
    AND b.process_name = LOWER(b.OriginalFileName)
'''
)

In [33]:
Hook_Injection.count()

4

In [34]:
Hook_Injection.show()

+------------------+--------------------+--------------+----------------------+------------------+
|         host_name| process_parent_name|  process_name|process_granted_access|user_reporter_name|
+------------------+--------------------+--------------+----------------------+------------------+
|win10.marvel.local|      injectproc.exe|   notepad.exe|               2097151|            SYSTEM|
|win10.marvel.local|processhollowing.exe|   svchost.exe|               2097151|            SYSTEM|
|win10.marvel.local|      injectproc.exe|   notepad.exe|               2097151|            SYSTEM|
|win10.marvel.local|             cmd.exe|powershell.exe|               2097151|            SYSTEM|
+------------------+--------------------+--------------+----------------------+------------------+



In [44]:
Hook_Injection = spark.sql(
'''
SELECT 
    b.host_name,
    b.process_parent_name, -- Might be work taking out "svchost" AND process_name "rundll32.exe". Need to see if this is a windows function somewhere
    b.process_name,
    a.process_granted_access,
    a.user_reporter_name
FROM sysmon_events b
JOIN sysmon_events a
    ON a.process_target_guid = b.process_guid
    AND a.event_id = 10
    AND a.process_granted_access = "2097151"
    AND b.process_parent_name = a.process_name
    AND a.process_name != b.process_name
JOIN sysmon_events c
    ON b.process_guid = c.process_guid
    AND c.event_id = 13
    AND c.process_guid = a.process_target_guid
    AND c.process_name = a.process_target_name 
    AND (substring_index(c.registry_key_path, '\\\\', -1)) = b.process_name
JOIN sysmon_events d
    ON d.process_guid = a.process_target_guid
    AND d.event_id = 7
    AND d.module_loaded = a.process_target_path
    AND LOWER(d.OriginalFileName) = (substring_index(c.registry_key_path, '\\\\', -1))
    AND d.OriginalFileName = b.OriginalFileName
WHERE
     b.event_id = 1
    AND b.process_name = LOWER(b.OriginalFileName)
    AND NOT b.process_name = "rundll32.exe" WHERE NOT b.process_parent_name = "svchost.exe"
'''
).show()

ParseException: '\nmismatched input \'WHERE\' expecting <EOF>(line 30, pos 44)\n\n== SQL ==\n\nSELECT \n    b.host_name,\n    b.process_parent_name, -- Might be work taking out "svchost" AND process_name "rundll32.exe". Need to see if this is a windows function somewhere\n    b.process_name,\n    a.process_granted_access,\n    a.user_reporter_name\nFROM sysmon_events b\nJOIN sysmon_events a\n    ON a.process_target_guid = b.process_guid\n    AND a.event_id = 10\n    AND a.process_granted_access = "2097151"\n    AND b.process_parent_name = a.process_name\n    AND a.process_name != b.process_name\nJOIN sysmon_events c\n    ON b.process_guid = c.process_guid\n    AND c.event_id = 13\n    AND c.process_guid = a.process_target_guid\n    AND c.process_name = a.process_target_name \n    AND (substring_index(c.registry_key_path, \'\\\\\', -1)) = b.process_name\nJOIN sysmon_events d\n    ON d.process_guid = a.process_target_guid\n    AND d.event_id = 7\n    AND d.module_loaded = a.process_targe

In [20]:
AND (process_granted_access & 5184) == 5184 -- 5184 is decimal for 0x1440. The minimal privileges you need to access process handle
    AND a.process_granted_access = "2047999"

SyntaxError: invalid syntax (<ipython-input-20-29f0fe263280>, line 1)

In [ ]:
# https://github.com/secrary/InjectProc

In [26]:
2047999 & 40 == 40

True